STEP 1 : Download all necessary tools

In [38]:
from gym import Env
from gym.spaces import Box, Discrete
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from PIL import Image
from collections import deque
from tqdm import tqdm
import random
import pytesseract
import pydirectinput
from matplotlib import pyplot as plt
from mss import mss
import pyautogui
import time
import cv2


In [32]:
# Configuração do ambiente
# env = gym.make('CartPole-v1')
# state_size = env.observation_space.shape[0]
# action_size = env.action_space.n

# Parâmetros para o DQN
learning_rate = 0.001
discount_rate = 0.95
exploration_rate = 1.0
max_exploration_rate = 1.0
min_exploration_rate = 0.01
exploration_decay_rate = 0.995
total_episodes = 1000
batch_size = 64

In [33]:
# Verificar se a GPU está disponível
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [34]:
class DinoAI(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low = 0, high = 255, shape = (1,83,100), dtype = np.uint8)
        self.action_space = Discrete(3)
        #Capture enviromment
        self.cap = mss()
        self.game_location = {'top':350, 'left': 0,'width':250, 'height': 150}
        self.gameOver_location = {'top':330, 'left': 310,'width':330, 'height': 25}
     
    def set_env(self):
        # Pressiona a tecla Windows
        pyautogui.press('win')
        time.sleep(1)
        # Escreve "Google Chrome"
        pyautogui.write('Google Chrome')
        time.sleep(1)
        # Aperta Enter
        pyautogui.press('enter')
        time.sleep(2)
        pyautogui.hotkey('ctrl', 'l')
        time.sleep(1)
        # Escreve "chrome://dino" na URL
        pyautogui.write('chrome://dino')
        pyautogui.press('enter')
        time.sleep(1)
        # Pressiona "Windows + seta para a esquerda"
        pyautogui.hotkey('win', 'left')
        time.sleep(1)
        pyautogui.hotkey('alt', 'tab')
        time.sleep(1)
        pyautogui.hotkey('win', 'right')
        time.sleep(1)
        pyautogui.hotkey('alt', 'tab')
        time.sleep(1)
        pyautogui.press('space')
        time.sleep(2)

    def get_gameOver(self):

        #time.sleep(5)
        gameOver_snip = np.array(self.cap.grab(self.gameOver_location))[:,:,:3]
        gray_gameOver = cv2.cvtColor(gameOver_snip, cv2.COLOR_BGR2GRAY)
        #print("forma:",gray_gameOver)

        return gray_gameOver

    def get_observation(self):
        #Start to set the enviroment
        #self.set_env()
        game_snip = np.array(self.cap.grab(self.game_location))[:,:,:3]
        gray = cv2.cvtColor(game_snip, cv2.COLOR_BGR2GRAY) 
        resized = cv2.resize(gray, (100,83))
        channel = np.reshape(resized, (1,83,100))

        return channel
    
    def get_done(self):

        gameOver_string = ['GAME','GAHE']
        done = False
        res = pytesseract.image_to_string(self.get_gameOver())[:4]
        if res in gameOver_string:
            done = True

        return done
    

    def step(self,action):

        if action == 1 :
            pyautogui.press('space')
        elif action == 2:
            pyautogui.press('down')

        new_state = self.get_observation()

        done = self.get_Done()

        reward = 1

        info = {}

        return new_state, reward, done, info




    def play_game(self):
        #self.set_env()
        done = False
        while(not done):
            new_state, reward, done, _ = self.step()

            print(done)
        
        
    def reset(self):
        pyautogui.press('space')


    # def render(self):
    #     pass

In [35]:
env = DinoAI()

In [43]:
# Rede neural para Deep Q-learning Model
# class DQN(nn.Module):
#     def __init__(self, state_size, action_size = 3):
#         super(DQN, self).__init__()
#         self.fc1 = nn.Linear(state_size, 24)
#         self.fc2 = nn.Linear(24, 24)
#         self.fc3 = nn.Linear(24, action_size)
    
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x
#     # Função para escolher a ação


def conv2d_size_out(size, kernel_size=5, stride=2):
    return (size - (kernel_size - 1) - 1) // stride + 1


class DQN(nn.Module):
    def __init__(self, img_height, img_width, num_actions = 3):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=2)
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(img_width)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(img_height)))
        linear_input_size = convw * convh * 32
        self.fc1 = nn.Linear(linear_input_size, 256)
        self.fc2 = nn.Linear(256, num_actions)  # Saída tem dimensão de num_actions

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # Flatten da saída da última camada convolucional
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [44]:
state_size = env.game_location['width'] * env.game_location['height']
action_size = 3
model = DQN(state_size, action_size).to(device)  # Mover o modelo para a GPU
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

memory = deque(maxlen=2000)

RuntimeError: Trying to create tensor with negative dimension -449664: [256, -449664]

In [ ]:
def choose_action(state, exploration_rate):
    if np.random.rand() <= exploration_rate:
        return random.randrange(action_size)
    state = torch.FloatTensor(state).to(device)
    with torch.no_grad():
        q_values = model(state)
    return np.argmax(q_values.cpu().numpy())

# Função para treinar o modelo
def train_model(batch_size):
    minibatch = random.sample(memory, batch_size)
    for state, action, reward, next_state, done in minibatch:
        state = torch.FloatTensor(state).to(device)
        next_state = torch.FloatTensor(next_state).to(device)
        reward = torch.FloatTensor([reward]).to(device)
        done = torch.FloatTensor([done]).to(device)

        target = reward
        if not done:
            target = reward + discount_rate * torch.max(model(next_state))

        target_f = model(state)
        target_f = target_f.clone()
        target_f[0][action] = target

        optimizer.zero_grad()
        loss = criterion(model(state), target_f)
        loss.backward()
        optimizer.step()


# Loop principal de treinamento
for e in range(total_episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        action = choose_action(state, exploration_rate)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print(f"Episode: {e+1}/{total_episodes}, Score: {time}, Exploration rate: {exploration_rate:.2f}")
            break
        if len(memory) > batch_size:
            train_model(batch_size)
    exploration_rate *= exploration_decay_rate
    exploration_rate = max(min_exploration_rate, exploration_rate)

# Salvar o modelo
torch.save(model.state_dict(), 'dqn_model.pth')